<a href="https://colab.research.google.com/github/sivanv-unbxd/colab-templates/blob/A2C_export_template/Pepper_X_A2C_Import_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook contains all the information and utilities required for performing an Import of catalog or other Product information into PIM.

### All Packages Imports & referneces

In [50]:
!pip install boto3
!pip install oauth2
!pip install swagger-client@git+https://github.com/sivanv-unbxd/a2c-sdk-pim.git@59b80b7c7f80470480e23971b9b3d69ecd91ae63


In [61]:
#@title Imports of Libraries
import os
import json
import requests
import time
import pandas as pd
import boto3
import random
import oauth2
from botocore.exceptions import ClientError
from traceback import print_exc

### Resource Utilities

In [62]:
def get_pim_app_domain():
    return "http://pimqa-apps.unbxd.io/pim/"

def get_pim_domain():
    return "http://pimqa.unbxd.io/"

def get_a2c_domain():
    return "https://api.api2cart.com/"
def get_a2c_api_key():
    return "8a71acf63ad0192c8d8f18febd073471"

def get_pepperx_domain():
    # return "https://pimqa.unbxd.io/pepperx/"
    return "https://sivanv.in.ngrok.io/app/"

### PIM SDK Utilities

In [63]:
import requests

import json
from math import floor

class Import(object):
    def __init__(self, pim_prop):
        self.pim_prop = pim_prop

    def import_to_pim(self, file_path):
        url = "{}/v1/imports".format(get_pim_domain())

        payload = dict()
        payload["url"] = file_path

        headers = {
            'Content-Type': "application/json",
            'Authorization': self.pim_prop.api_key,
        }
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        if response.status_code != 200:
            logger.error(
                "Pim Product Pull failed due non 200 status data:  {} status_code {} ".format(
                    response.status_code, response.text
                )
            )
            raise ValueError("Pim Product Pull failed due non 200 status " + response.text)
        if "data" not in response.json() or "import_id" not in response.json()["data"]:
            raise ValueError("Pim Product Pull failed due to data expectation")

        return response.json()["data"]["import_id"]


class PimProductList(object):
    def __init__(
            self, api_key, reference_id=None, properties=[], group_by_parent=None, parent_id=None, q=None,
            cache_count=10, app_id=None
    ):
        self.api_key = api_key
        self.properties = properties
        self.group_by_parent = group_by_parent
        self.q = q
        self.cache_count = cache_count
        self.cache = []
        self.reference_id = reference_id
        self.app_id = app_id
        self.parent_id = parent_id

    def __iter__(self):
        self.n = 0
        response = self.get(count=10)

        if "data" not in response or "total" not in response["data"]:
            raise ValueError("Invalid response returned by PIM")
        self.max = response["data"]["total"]
        return self

    def __next__(self):
        if self.n < self.max:
            index = self.n % self.cache_count
            if index == 0:
                page = floor(self.n / self.cache_count) + 1
                response = self.get(count=self.cache_count, page=page)
                if "data" not in response or "products" not in response["data"]:
                    raise ValueError("Invalid response returned by PIM")
                products = response["data"]["products"]
                self.cache = products
            self.n += 1
            if len(self.cache) > index:
                return self.cache[index]
        else:
            raise StopIteration

    def get(self, count=10, page=1):
        url = "{}v1/products".format(get_pim_app_domain())
        headers = {
            'Content-Type': "application/json",
            'Authorization': self.api_key,
        }
        req = {
            "page": page,
            "count": count,
            "groupByParent": self.group_by_parent,
            "properties": self.properties,
            "q": self.q,
            "referenceId": self.reference_id,
            "parentId": self.parent_id
        }
        logger.info("Requesting URL {}".format(url))
        logger.info("Request for PIM products : {}".format(str(json.dumps(req))))
        response = requests.post(url, headers=headers, json=req)
        if response.status_code != 200:
            raise PimProductException(
                "Pim Product Pull failed due non {} response body {} - reason {} ==> Request Object >> {}".format(
                    response.status_code, response.text, response.reason, str(req)
                )
            )
        if "data" not in response.json() or "products" not in response.json()["data"]:
            raise PimProductException("Pim Product Pull failed due to data expectation")

        return response.json()





### Pepper-X SDK utilities

In [64]:
import requests
import json

class App(object):
    def __init__(self, app_id="", app_name=""):
        self.app_id= app_id
        self.app_name = app_name
        self.get(app_id, app_name)

    def create(self, app_id, name, credentials={}):
        try:
            url = f"{get_pepperx_domain()}api/v1/app_data/"
            payload = json.dumps({
                "app_id": app_id,
                "app_name": name,
                "app_creds" : credentials
            })
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            data =response.text
            if response.status in [200, 201]:
                return data
            else:
                print(response)
                return None

        except Exception as e:
            print_exc()
            print(e)

    def get(self, app_id, app_name=""):
            try:
                url = f"{get_pepperx_domain()}api/v1/app_data/"
                payload = json.dumps({
                    "app_id": app_id,
                })
                headers = {
                    'Content-Type': 'application/json'
                }
                response = requests.request("GET", url, headers=headers, data=payload)
                data =response.text
                if response.status_code in [200, 201]:
                    app_data =  json.loads(data)
                    app_data = app_data["data"]
                    self.app_creds = app_data["app_data"]["app_creds"]
                    self.app_name = app_data["app_data"]["app_name"]
                    self.app_id = app_data["app_data"]["app_id"]
                    return app_data
                else:
                    return None

            except Exception as e:
                print_exc()
                print(e)


class AppUser(object):
    def __init__(self, app_id, identifier):
        self.app_id= app_id
        self.identifier= identifier



    def create(self, credentials={}, pim_creds ={}):
        try:
            url = f"{get_pepperx_domain()}api/v1/app_user_data/"
            payload = json.dumps({
                "app_id": self.app_id,
                "identifier": self.identifier,
                "user_creds" : credentials,
                "pim_creds": pim_creds
            })
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            data =response.text
            if response.status_code in [200, 201]:
                app_data =  json.loads(data)
                app_data = app_data["data"]
                self.app_user_creds = app_data["app_user"]["app_creds"]
                self.pim_creds = app_data["app_user"].get("pim_creds", None)
            else:
                return data


        except Exception as e:
            print_exc()
            print(e)

    def get(self):
        try:
            url = f"{get_pepperx_domain()}api/v1/app_user_data/"
            payload = json.dumps({
                "app_id": self.app_id,
                "identifier" : self.identifier
            })
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.request("GET", url, headers=headers, data=payload)
            data = response.text
            if response.status_code in [200, 201]:
                app_data = json.loads(data)
                app_data = app_data["data"]
                return app_data
            else:
                return None
        except Exception as e:
            print_exc()
            print(e)

class AppUserPIM(object):
    def __init__(self, api_key=""):
        self.api_key = api_key

    def get(self):
        try:
            url = f"{get_pepperx_domain()}api/v1/app_user_pim_data/"

            payload = json.dumps({
                "api_key": self.api_key
            })
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.request("GET", url, headers=headers, data=payload)
            data =response.text
            print("User creds & PIM Creds")
            print(response.text)
            print("API Status")
            print(response.status_code)
            if response.status_code in [200, 201]:
                app_data = json.loads(data)
                app_data = app_data["data"]
                # self.app_creds = app_data["credentials"]
                # self.app_name = app_data["app_data"]["name"]
                # self.app_id = app_data["app_data"]["label"]
                return app_data
            else:
                return None

        except Exception as e:
            print_exc()
            print(e)

### Import 1-D to n-D Transformer

In [65]:
from __future__ import print_function
import json
import logging
from datetime import datetime, timezone

import time
import swagger_client
import re
from swagger_client.rest import ApiException

from pprint import pprint

class Transformer:
    def __init__(self, config):

        self.config = config

    def _update_product(self,product, key, data):
        if type(data) is dict:
            product.update(data)
        else:
            product[key] = data

    def transform_child(self, product):
        return product

    def transform(self, product, a2c_api_key= "", a2c_store_key= "", category_list= ""):
        changed_product = {}
        image_object = {}
        attribute_object = {}
        attribute_list = []

        for fieldname in product:
            if fieldname in self.config:
                transformer = self.config[fieldname]
                changed_product[transformer['key']] = product.get(fieldname)
                if 'helper' in transformer:
                    if transformer["helper"] == "handle_categories":
                        helper_func = getattr(self, "handle_categories")
                        helper_params = {"category_list": category_list, "a2c_api_key": a2c_api_key, "a2c_store_key":a2c_store_key}
                        self._update_product(changed_product, transformer["key"],
                                             helper_func(product, fieldname, helper_params))
                    else:
                        helper_func = getattr(self, transformer['helper'])
                        self._update_product(changed_product, transformer['key'], helper_func(product, fieldname))

            elif re.search("image_url",fieldname):
                helper_func = getattr(self, "handler_image_urls")
                self._update_product(image_object, fieldname, helper_func(product, fieldname))
                # print(changed_product)

            #TODO Change variant_option to OP
            elif re.search("variant_option", fieldname):
                helper_func = getattr(self, "handle_attribute")
                self._update_product(attribute_object, fieldname, helper_func(product, fieldname))
                if all([len(list(attribute_object.keys())) != 0, attribute_object not in attribute_list]):
                    attribute_list.append(attribute_object.copy())
            # else:
            #     if product[fieldname] not in [None, 'none', 'None']:
            #         changed_product[fieldname] = product[fieldname]
        remove_index = []
        for dict_obj_index in range(len(attribute_list)):
            if len(list(attribute_list[dict_obj_index].keys())) != 3:
                remove_index.append(dict_obj_index)
        for index in remove_index:
            attribute_list.pop(index)
        changed_product["attributes"] = attribute_list or None
        changed_product["image_object"] = image_object or None

        return changed_product

    def handler_datetime(self,product, key):
        try:
            dt = datetime.fromisoformat(product[key])
        except ValueError:
            dt = datetime.utcnow()
        dt = dt.astimezone(tz=timezone.utc)
        return dt.strftime('%Y-%m-%dT%H%M%SZ')


    def handler_string(self,product, key):
        return str(product[key]).strip()


    def handler_id(self,product, key):
        return str(product[key])


    def handler_special_price(self,product, key):
        if product["avail_sale"] == 'true':
            return product[key]


    def handle_list_to_string(self,product, key):

        description = ""
        for string in product[key]:
            description = description + " " + str(string)

        unwanted_chars = ["<p>", "</p>"]
        for char in unwanted_chars:
            description = description.replace(char, "")




        return description


    def handle_url(self,product, key):

        if 'https://' in product[key]:
            product[key] = product[key][8:]
        elif 'http://' in product[key]:
            product[key] = product[key][7:]

        return product[key]


    def handle_categories(self,product, key, parameters):
        category_list = parameters["category_list"]
        a2c_api_key = parameters["a2c_api_key"]
        a2c_store_key = parameters["a2c_store_key"]
        build_category_tree_instance = BuildCategoryTree(category_list, a2c_api_key, a2c_store_key)
        category_full_path_map = build_category_tree_instance.build_categories()
        category_ids = build_category_tree_instance.getCategoryIds(product[key])
        result = ','.join(category_ids)

        return result

    def handler_image_urls(self,product, key):


        temp_key = key
        split_list = temp_key.split('_')
        counter = split_list[2]
        prop_name = split_list[-1]
        img_props = {}
        if prop_name == "order":
            img_props[counter + "-" + "position"] = product[key]

        elif prop_name == "path":
            img_props[counter + "-" + "url"] = product[key][0]
        elif prop_name == "type":

            img_props[counter + "-" +"type"] = product[key].split('_')[-1].lower()
        elif prop_name == "name":
            img_props[counter + "-" + "image_name"] = product[key]



        return img_props

    def handle_attribute(self,product, key):
        temp_key = key
        attribute_object = {}
        split_list = temp_key.split('_')

        if split_list[-1] == "name":
            attribute_object["attribute_name"] = split_list[2]
            attribute_object["attribute_value"] = product[key]
        elif split_list[-1] == "price":
            if len(split_list) == 4:
                attribute_object["attribute_price"] = product[key]
        return attribute_object

### File Operation Utilities


In [66]:
import os
import csv
import json
import zipfile
import tempfile
from time import time
import logging
import boto3
from botocore.exceptions import ClientError


def create_tmp_file(data):
    filename = tempfile.NamedTemporaryFile(delete=False).name
    with open(filename, "wb") as f:
        f.write(data)
    return filename


def write_csv_file(data, delimiter="\t"):
    """
        Method write array data to the file and return the file name
    :param data: (list)- Array of array data i.e is supposed to be written to the file.
    :param delimiter:
    :return:
    """
    filename = tempfile.NamedTemporaryFile(delete=False).name
    with open(filename, 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=delimiter)
        csvwriter.writerows(data)
    return filename


def read_csv_file(data_file, delimiter=","):
    with open(data_file, 'r') as text:
        csv_reader = csv.reader(text, delimiter=delimiter)
        for row in csv_reader:
            yield row


def read_csv_stream(stream, delimiter="\t"):
    csv_reader = csv.reader(iter(stream), delimiter=delimiter)
    for row in csv_reader:
        yield row


def compress_file_to_zip(zip_file_name, local_file_path, arcname):
    """
        Function to compress a file to a zip file.
    :param zip_file_name: (string) - Zipped file name
    :param local_file_path: (string) - File's path on the system.
    :param arcname: (string)  File name without the directory names
    :return: None
    """

    dir_name = os.path.dirname(zip_file_name)
    # check if the directory is not there
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    print("### In zipping file from {}  to {}",local_file_path, zip_file_name)
    with zipfile.ZipFile(
        zip_file_name, 'w', zipfile.ZIP_DEFLATED
    ) as zipped_items:
        zipped_items.write(local_file_path, arcname=arcname)


def write_to_file(data, file_path, mode="w", is_json=True):
    """
        Function to write json data fil
    :param data:
    :param file_path: (string) - absolute file path
    :return:
    """
    dir_name = os.path.dirname(file_path)
    # check if the directory is not there
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    print("File path in os == > ", file_path)
    try:
        with open(file_path, mode) as outfile:
            if is_json:
                json.dump(data, outfile)
            else:
                outfile.write(data)
    except Exception as e:
        raise Exception("Exception {}".format(str(e)))


def write_feed_file(file_dir, products, app_name=""):
    current_time = round(time())
    file_name = '{app_name}_to_PIM_feed_{timestamp}.json'.format(
        timestamp=current_time, app_name = app_name
    )
    zip_file_name = '{app_name}_to_PIM_feed_{timestamp}.zip'.format(
        timestamp=current_time, app_name = app_name
    )
    file_path = os.path.join(file_dir, file_name)
    with open(file_path, 'w') as f:
        f.write(json.dumps(products))
    zip_file_path = os.path.join(file_dir, zip_file_name)
    compress_file_to_zip(zip_file_path, file_path, file_name)
    return file_name

def extract_zip_file(zip_file_with_path):
    # opening the zip file in READ mode
    with zipfile.ZipFile(zip_file_with_path, 'r') as zip:
        # printing all the contents of the zip file
        files_in_zip = []
        for files in zip.filelist:
            if '.' in files.filename:
                files_in_zip.append(files.filename)
        zip.printdir()
        # extracting all the files
        print('Extracting all the files now...')
        zip.extractall()
        return files_in_zip


def handle_dict(dict_object, params):
    # TODO handle methods
    # Params: [accepted_keys, ignore_keys, required_key,prefix, suffix, op_type]
    op_dict = {}
    item_key = ""
    for key, value in dict_object.items():
        if "accepted_keys" in params and key in params["accepted_keys"]:
            # if all(item_key in sub for sub in [dict_object[key], params["accepted_keys"]]):
                op_dict[key] = dict_object[key]
        else:
            # TODO
            print("Handle all param types")
            #op_dict = dict_object

    return op_dict


def upload_to_s3(file_path, filename, bucket, object_name=None):

        """Upload a file to an S3 bucket

        :param file_name: File to upload
        :param bucket: Bucket to upload to
        :param object_name: S3 object name. If not specified then file_name is used
        :return: True if file was uploaded, else False
        """
        region = os.environ['aws_region']
        key = "app-uploads/" + filename
        # If S3 object_name was not specified, use file_name
        if object_name is None:
            object_name = filename
        s3 = boto3.resource(
            service_name='s3',
            region_name=region,
            aws_access_key_id=os.environ['aws_access_key_id'],
            aws_secret_access_key=os.environ['aws_secret_access_key']
        )

        try:
            s3.Bucket(bucket).upload_file(Filename=file_path+filename, Key=key)
        # Upload the file
        # s3_client = boto3.client('s3')
        # try:
        #     response = s3_client.upload_file(filename_with_path, bucket, object_name)
            url = f"https://{bucket}.s3.{region}.amazonaws.com/{key}"
        except ClientError as e:
            logging.error(e)
            return False
        return url
def upload_json(req_data, input_file_name):

  named_tuple = time.localtime() # get struct_time
  time_string = time.strftime("-%m-%d-21-%H-%M-%S", named_tuple)  
  s3 = boto3.resource('s3')
  bucket = "unbxd-pim-ui"
  file_name = f'{input_file_name}{time_string}.json'
  region = os.environ['aws_region']
  aws_access_key_id=os.environ['aws_access_key_id']
  aws_secret_access_key=os.environ['aws_secret_access_key']
  key = str("app-uploads/" + file_name)
  s3 = boto3.resource(
      service_name='s3',
      region_name=region,
      aws_access_key_id=aws_access_key_id,
      aws_secret_access_key=aws_secret_access_key
  )
  s3object = s3.Object(bucket, key)

  s3object.put(
      Body=(bytes(json.dumps(req_data).encode('UTF-8')))
  )
  url = f"https://{bucket}.s3.{region}.amazonaws.com/{key}"
  print(url)
  return url

### Category Module

In [67]:
from __future__ import print_function
import time
import swagger_client
from swagger_client.rest import ApiException
from pprint import pprint


class BuildCategoryTree:
    def __init__(self, catList, a2c_api_key, a2c_store_key):
        self.catList = catList
        self.categoryTreePath = dict()
        self.a2c_api_key = a2c_api_key
        self.a2c_store_key = a2c_store_key

    def build_categories(self):

        dictfilt = lambda x, y: dict([(i, x[i]) for i in x if i in set(y)])
        categoriesList = []
        cat_keys = ["id", "name", "parent_id"]
        for cat in self.catList:
            categoriesList.append(dictfilt(cat, cat_keys))

        self.categoryTreePath = self.buildCategoryTree(categoriesList)
        return self.categoryTreePath

    def buildCategoryTree(self, catData):
        categoryFullPath = self.categoryTreePath
        for cat in catData:
            if (cat["parent_id"] == '0'):
                categoryFullPath[cat["id"]] = cat["name"]
        for cat in catData:
            try:
                if (cat["parent_id"] == '0'):
                    categoryFullPath[cat["id"]] = cat["name"]
                elif (categoryFullPath[cat["parent_id"]]):
                    categoryFullPath[cat["id"]] = categoryFullPath[cat["parent_id"]] + ">" + cat["name"]
            except Exception as e:
                print(e)

        return categoryFullPath
    def getCategoryIds(self, cats):
        catArr = []
        if (cats != None):
            for cat in cats:
                catId = self.getCategoryId(cat)
                catArr.append(catId)
        print("$$$$$ updated categories", catArr)

        return catArr


    def getCategoryId(self, catName):
        try:
            catId = list(self.categoryTreePath.keys())[list(self.categoryTreePath.values()).index(catName)]
        except Exception as e:
            print("Need to create", catName)
            cat_path = catName.split(">")
            parent_id = 0
            name = ""
            for cat_node in cat_path:
                name += cat_node
                print("!!!!!inside for loop cat_node   ", cat_node, " ----- full cat path is ==> ", name)
                try:
                    print("checking cat id for ", name)
                    parent_id = list(self.categoryTreePath.keys())[list(self.categoryTreePath.values()).index(name)]
                except  Exception as e:
                    print("creating category with name as ==> ", cat_node, " and parent id as ==>", parent_id)
                    configuration = swagger_client.Configuration()
                    configuration.api_key['api_key'] = self.a2c_api_key
                    configuration.api_key['store_key'] = self.a2c_store_key
                    category_create_instance = swagger_client.CategoryApi(swagger_client.ApiClient(configuration))
                    name = cat_node  # str | Defines category's name that has to be added
                    parent_id = parent_id  # str | Adds categories specified by parent id (optional)
                    category_create_response = category_create_instance.category_add(name, parent_id=parent_id)
                    pprint(category_create_response)
                    print("****** created category is ==>", str(name))
                    dictionary = {"id":category_create_response.result.category_id, "name": name, "parent_id": parent_id}
                    parent_id = category_create_response.result.category_id
                    self.categoryTreePath = self.buildCategoryTree([dictionary])
                    catId = parent_id
                name += ">"
        return catId

### App Schema 

In [68]:
product_schema={
"id": {
"key": "id",
"helper": "handler_id"
},
"image_urls": {
"key": "image_urls",
"helper": "handler_image_urls"
},
"description": {
"key": "description",
"helper": "handle_list_to_string"
},
"url": {
"key": "url"
},
"create_at": {
"key": "created_at"
},
"name": {
"key": "name"
},
"u_model": {
"key": "model"
},
"u_sku": {
"key": "sku"
},
"price": {
"key": "price"
},
"old_price": {
"key": "old_price"
},
"special_price_value": {
"key": "special_price",
"helper": "handler_special_price"
},
"cost_price": {
"key": "cost_price"
},
"special_price_create": {
"key": "sprice_create",
"helper": "handler_special_price"
},
"special_price_modified": {
"key": "sprice_modified",
"helper": "handler_special_price"
},
"special_price_expire": {
"key": "sprice_expire",
"helper": "handler_special_price"
},
"tier_prices": {
"key": "tier_prices"
},
"group_prices": {
"key": "group_prices"
},
"avail_view": {
"key": "available_for_view"
},
"avail_sale": {
"key": "available_for_sale"
},
"weight": {
"key": "weight"
},
"weight_unit": {
"key": "weight_unit"
},
"short_description": {
"key": "short_description"
},
"warehouse_id": {
"key": "warehouse_id"
},
"allow_backorders": {
"key": "backorder_status"
},
"quantity": {
"key": "quantity"
},
"is_downloadable": {
"key": "downloadable"
},
"wholesale_price": {
"key": "wholesale_price"
},
"manufacturer": {
"key": "manufacturer"
},
"categories": {
"key": "categories_ids",
"helper": "handle_categories"
},
"tax_class_id": {
"key": "tax_class_id"
},
"type": {
"key": "type"
},
"meta_title": {
"key": "meta_title"
},
"meta_keywords": {
"key": "meta_keywords"
},
"meta_description": {
"key": "meta_description"
},
"lang_id": {
"key": "lang_id"
},
"stores_ids": {
"key": "stores_ids"
},
"viewed_count": {
"key": "viewed_count"
},
"ordered_count": {
"key": "ordered_count"
},
"attribute_set_name": {
"key": "attribute_set_name"
},
"attribute_name": {
"key": "attribute_name"
},
"shipping_template_id": {
"key": "shipping_template_id"
},
"condition": {
"key": "condition"
},
"listing_duration": {
"key": "listing_duration"
},
"listing_type": {
"key": "listing_type"
},
"payment_methods": {
"key": "payment_methods"
},
"return_accepted": {
"key": "return_accepted"
},
"shipping_details": {
"key": "shipping_details"
},
"paypal_email": {
"key": "paypal_email"
},
"seller_profiles": {
"key": "seller_profiles"
},
"best_offer": {
"key": "best_offer"
},
"package_details": {
"key": "package_details"
},
"sales_tax": {
"key": "sales_tax"
},
"barcode": {
"key": "barcode"
},
"upc": {
"key": "upc"
},
"ean": {
"key": "ean"
},
"isbn": {
"key": "isbn"
},
"specifics": {
"key": "specifics"
},
"image_url": {
"key": "image_url"
},
"image_name": {
"key": "image_name"
},
"avail_from": {
"key": "avail_from"
},
"tags": {
"key": "tags"
},
"clear_cache": {
"key": "clear_cache"
},
"gtin": {
"key": "gtin"
},
"taxable": {
"key": "taxable"
},
"visible": {
"key": "visible"
},
"status": {
"key": "status"
},
"product_class": {
"key": "product_class"
},
"retail_price": {
"key": "retail_price"
},
"increase_quantity": {
"key": "increase_quantity"
},
"reduce_quantity": {
"key": "reduce_quantity"
},
"reserve_quantity": {
"key": "reserve_quantity"
},
"manage_stock": {
"key": "manage_stock"
},
"manufacturer_id": {
"key": "manufacturer_id"
},
"in_stock": {
"key": "in_stock"
},
"seo_url": {
"key": "seo_url"
},
"report_request_id": {
"key": "report_request_id"
},
"disable_report_cache": {
"key": "disable_report_cache"
},
"reindex": {
"key": "reindex"
}
}

variant_product_schema={
"parent_id": {
"key": "product_id",
"helper": "handler_id"
},
"image_urls": {
"key": "image_urls",
"helper": "handler_image_urls"
},
"name": {
"key": "name"
},
"u_model": {
"key": "model"
},
"u_sku": {
"key": "sku"
},
"barcode": {
"key": "barcode"
},
"price": {
"key": "price"
},
"cost_price": {
"key": "cost_price"
},
"attributes": {
"key": "attributes"
},
"description": {
"key": "description",
"helper": "handle_list_to_string"
},
"special_price": {
"key": "special_price"
},
"sprice_create": {
"key": "sprice_create"
},
"sprice_modified": {
"key": "sprice_modified"
},
"sprice_expire": {
"key": "sprice_expire"
},
"avail_view": {
"key": "available_for_view"
},
"avail_sale": {
"key": "available_for_sale"
},
"weight": {
"key": "weight"
},
"weight_unit": {
"key": "weight_unit"
},
"short_description": {
"key": "short_description"
},
"warehouse_id": {
"key": "warehouse_id"
},
"quantity": {
"key": "quantity"
},
"create_at": {
"key": "created_at"
},
"manufacturer": {
"key": "manufacturer"
},
"tax_class_id": {
"key": "tax_class_id"
},
"meta_title": {
"key": "meta_title"
},
"meta_keywords": {
"key": "meta_keywords"
},
"meta_description": {
"key": "meta_description"
},
"url": {
"key": "url",
"helper": "handle_url"
},
"store_id": {
"key": "store_id"
},
"lang_id": {
"key": "lang_id"
},
"clear_cache": {
"key": "clear_cache"
},
"taxable": {
"key": "taxable"
}
}

### Product Exporter

In [69]:
from __future__ import print_function
import time
import swagger_client
from swagger_client.rest import ApiException
from pprint import pprint

import logging
import requests
import json
from traceback import print_exc

class ProductExporter(object):

    def __init__(self, api_key, reference_id):

        # TODO Call PAS/app.py-> getUserPropsFromAPIKey
        self.api_key = api_key
        self.reference_id = reference_id

        try:
            app_user_instance = AppUserPIM(self.api_key)
            app_data = app_user_instance.get()

            self.send_store_key = app_data["app_user"]["app_creds"]["store_key"]
            #self.receive_store_key = "b1d6ecaa3a33f918ca774eed03b29d6b"
            self.a2c_api_key = app_data["app_data"]["app_creds"]["a2c_api_key"]
            #self.a2c_api_key = "8a71acf63ad0192c8d8f18febd073471"
            #self.send_store_key = "af40d78830d767d23fefc362ddca4f31"
        except Exception as e:
            logger.error("Value not found in app database")
            print_exc()

        self.configuration = swagger_client.Configuration()
        self.configuration.api_key['api_key'] = self.a2c_api_key
        self.configuration.api_key['store_key'] = self.send_store_key

    def __del__(self):
        # Call the delete or garbage clear
        print("Do Cleanup")

    def main(self):
        self.pullPimProducts()
    # 1. Pulls products and variants from PIM

    def pullPimProducts(self):

        products_list = []

        #TODO Transform products
        transformer = Transformer(product_schema)
        variant_transformer = Transformer(variant_product_schema)
        counter = 1
        # logger.info("Update from PIM to Shopify have begun for user: {} with reference_id: {}".
        #             format(self.user.identifier, self.reference_id))
        try:
            start = 0
            count = 250
            params = 'id,parent_id,name,description'

            category_api_instance = swagger_client.CategoryApi(swagger_client.ApiClient(self.configuration))
            category_api_response = category_api_instance.category_list(start=start, count=count, params=params)
            category_list = [x.to_dict() for x in category_api_response.result.category]
            initial_product_list = []
            variant_products_list = []
            for product in PimProductList(self.api_key, self.reference_id, group_by_parent=True):
                counter = counter + 1
                # TODO Manage the product level cleanup and final expected custom channel format
                initial_product_list.append(product)
                transformed_product = transformer.transform(product, self.a2c_api_key, self.send_store_key,
                                                          category_list)
                products_list.append(transformed_product)
                if product["pimProductType"] == "PARENT":
                    #TODO transform and handle variants by calling the variant products
                    parent_id = product["id"]
                    for children_product in PimProductList(self.api_key, self.reference_id, group_by_parent=True, parent_id=parent_id):
                        transformed_child_product = variant_transformer.transform(children_product, self.a2c_api_key, self.send_store_key, category_list)
                        transformed_child_product["parent_id"] = parent_id
                        variant_products_list.append(transformed_child_product)



            print("fetched all products", len(products_list))


            #TODO make product export to A2C
            map_parent_id = {}
            for product in products_list:

                image_object = product.pop('image_object')
                #TODO Separate images and handle
                actual_product_add_dict = {key: value for key, value in product.items() if value}
                actual_product_add_dict.pop("url")
                old_id = actual_product_add_dict.pop("id")

                # TODO segregate properties for update and add
                update_list_properties = ["retail_price", "increase_quantity", "reduce_quantity", "reserve_quantity",
                                          "manage_stock", "manufacturer_id", "in_stock", "seo_url", "report_request_id",
                                          "disable_report_cache", "reindex"]

                product_update_dict = {}
                for property in update_list_properties:
                    if property in list(actual_product_add_dict.keys()):
                        product_update_dict[property] = actual_product_add_dict.pop(property)
                api_instance = swagger_client.ProductApi(swagger_client.ApiClient(self.configuration))
                #TODO product update or add
                #todo Check if product is already present
                find_value = product["name"]
                find_where = 'name'
                product_find_response = api_instance.product_find(find_value, find_where=find_where)
                if product_find_response.result.product is None:
                    body = swagger_client.ProductAdd(**actual_product_add_dict)  # ProductAdd |

                    product_add_response = api_instance.product_add(body)
                    product_id = product_add_response.result.to_dict()['product_id']
                    if product_id is not None:
                        map_parent_id[old_id] = product_id
                        product_update_response = api_instance.product_update(product_id, **product_update_dict)
                        # pprint(product_update_response)
                    # todo product image add

                        product_id = product_id  # str | Defines product id where the image should be added

                        list_of_image_objects = [{} for x in range(10)]
                        for image_props in image_object:
                            prop = image_props
                            split_list = prop.split("-")
                            counter = int(split_list[0])
                            list_of_image_objects[counter-1][split_list[1]] = image_object[image_props]

                        for actual_image_object in list_of_image_objects:
                            if "image_name" not in list(actual_image_object.keys()):
                                actual_image_object["image_name"] = "image"

                        for actual_image_object in list_of_image_objects:
                            if len(list(actual_image_object.keys())) == 4:
                                if actual_image_object["type"] == 'base':
                                    break
                                image_add_response = api_instance.product_image_add(product_id, **actual_image_object)
                                # pprint(image_add_response)
                                break

                    else:
                        print("Product add failed")
                else:
                    #TODO product update
                    id = product_find_response.result.product[0].id
                    map_parent_id[old_id] = id

                    product_update_response = api_instance.product_update(id, **product_update_dict)
                    # pprint(product_update_response)

            print(map_parent_id)
            #TODO Handle Variants
            variant_id_list = []
            for variants in variant_products_list:

                variant_image_object = variants.pop('image_object')
                list_of_variant_image_objects = [{} for x in range(10)]
                for image_props in variant_image_object:
                    prop = image_props
                    split_list = prop.split("-")
                    counter = int(split_list[0])
                    list_of_variant_image_objects[counter - 1][split_list[1]] = variant_image_object[image_props]

                for actual_variant_image_object in list_of_variant_image_objects:
                    if "image_name" not in list(actual_variant_image_object.keys()):
                        actual_variant_image_object["image_name"] = "image"



                new_parent_id = map_parent_id.get(variants.pop("parent_id"))

                # TODO Check for variants already present and update
                if new_parent_id:
                    variants["product_id"] = new_parent_id
                else:
                    print("Raise error in product")
                #TODO product_variant_add
                variant_add_instance = swagger_client.ProductApi(swagger_client.ApiClient(self.configuration))
                body = swagger_client.ProductVariantAdd(**variants)  # ProductVariantAdd |


                variant_add_response = variant_add_instance.product_variant_add(body)
                # pprint(variant_add_response)
                variant_id = variant_add_response.result.product_variant_id
                variant_id_list.append(variant_id)

                if variant_id:
                #TODO Product variant image add
                    variant_image_update_instance = swagger_client.ProductApi(swagger_client.ApiClient(self.configuration))
                    product_id = new_parent_id  # str | Defines product id where the variant image has to be added
                    product_variant_id = int(variant_id)  # int | Defines product's variants specified by variant id

                    for actual_variant_image_object in list_of_variant_image_objects:
                        if len(list(actual_variant_image_object.keys())) == 4:
                            if actual_variant_image_object["type"] == 'base':
                                break
                            product_variant_image_add_response = variant_image_update_instance.product_variant_image_add(product_id,
                                                                                                                         product_variant_id,
                                                                                                                         **actual_variant_image_object)
                            # pprint(product_variant_image_add_response)
                            break

            # TODO Update the celery thread to update job status
            print("Done exporting products")
        except Exception as e:
            print_exc()
            logger.error("Error : {} reason being {}".format("Could not fetch products from PIM", str(e)))
            raise e



### Starting Point for triggering the import

In [70]:
def trigger_product_export(api_key="", reference_id=""):
    product_exporter = ProductExporter(api_key=api_key, reference_id= reference_id)
    product_exporter.main()
    return {"data": "Finished Export task"}

In [71]:
trigger_product_export(api_key='60db34f8d000845784a82ec5',reference_id='60e73787149a4e5617f25aab')

User creds & PIM Creds
{"data": {"app_data": {"name": "Bigcommerce", "label": "Bigcommerce_PIM_A2C", "app_creds": {"client_id": "340ypj2sh5tldbeoqlud1b3e9q4grx1", "client_secret": "1bemkwyk9ccflbeb6hh4yk37f6nbo0r", "install_url": "https://login.bigcommerce.com/deep-links/marketplace/apps/17340", "redirect_uri": "https://pimapps.unbxd.io/setup/bigcommerce/oauth/?appId=BIGCOMMERCE_PIM_V1", "a2c_api_key": "8a71acf63ad0192c8d8f18febd073471"}}, "app_user": {"identifier": "BigCommerceTest2MyOrg", "app_creds": {"store_url": "https://store-57k01ggfwe.mybigcommerce.com", "client_id": "q537jzm9m3fzcuv4r437ndychv90zc7", "accessToken": "rpl18n00hcd4ot40ni8zowx3ktjp251", "context": "stores/57k01ggfwe", "org_app_id": "60db34f8d000845784a82ec6", "channel_id": "60c254c56bbefb000947bcc8", "adapter_id": "60db352ed000845784a82ec9", "store_key": "af40d78830d767d23fefc362ddca4f31"}, "pim_creds": {}}}}
API Status
200
fetched all products 1
{'111': '353'}
Done exporting products
Do Cleanup


{'data': 'Received Import task'}